# Lighthouse calibration

The aim is to change the coordinates of the lighthouse from arbitrary to "Arena" coordinates

In [4]:
# imports
%matplotlib inline
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [8]:
# find the data
data_root = Path('/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/calibration/old')
save_root = Path('/camp/lab/znamenskiyp/home/shared/projects/blota_onix_calibration/')
folders = dict(origin='origin_lighthouse',
              right='origin_100_0_lighthouse',
              bottom='origin_0_100_lighthouse',
              above='origin_z814_lightouse')

In [11]:
os.listdir(data_root / 'origin_lighthouse')

['video_two_2021-09-27T11_40_49.avi',
 'lighthouse_matrix2_origin_2021-09-27T11_40_48.dat',
 'lighthouse2_timestamps_2021-09-27T11_40_48.csv',
 'camera_two_timestamps_2021-09-27T11_40_48.csv',
 'lighthouse1_timestamps_2021-09-27T11_40_48.csv',
 'lighthouse_matrix1_origin_2021-09-27T11_40_48.dat']

In [3]:
# load floor plane data to check if it's flat
fpath = data_root / 'lighthouse_matrix2_floorplane_2021-09-28T10_26_20.dat'
data = np.fromfile(fpath, dtype='double')
data = data.reshape([-1,3])

FileNotFoundError: [Errno 2] No such file or directory: '/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/calibration/lighthouse_matrix2_floorplane_2021-09-28T10_26_20.dat'

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(2,2,1, projection='3d')
ax.scatter(data[:,0], data[:,1], data[:,2], c=data[:,2],s=5)
ax.view_init(elev=10., azim=145)
ax = fig.add_subplot(2,2,2, projection='3d')
ax.scatter(data[:,0], data[:,1], data[:,2],c=data[:,2], s=5)
ax.view_init(elev=10., azim=15)

ax = fig.add_subplot(2,2,3)
ax.scatter(data[:,0], data[:,1], c=data[:,2], s=10)
ax = fig.add_subplot(2,2,4)
ax.scatter(data[:,0], data[:,2], c=data[:,1], s=10)

In [ ]:
# same with a longer recording with the mouse
example_dir = data_root / '..' / 'tracking' / 'mouse_in_arena_210928'

data_files = ('lighthouse_matrix1_2021-09-28T14_17_34.dat',
              'lighthouse_matrix2_2021-09-28T14_17_34.dat')

matrix = []
for fname in data_files:
    data = np.fromfile(example_dir/fname, dtype='double')
    data = data.reshape([-1,3])
    print(data.shape)
    matrix.append(data)
data = np.vstack(matrix)
data.shape

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(2,2,1, projection='3d')
ax.scatter(data[:,0], data[:,1], data[:,2], c=data[:,2],s=5)
ax.view_init(elev=10., azim=145)
ax = fig.add_subplot(2,2,2, projection='3d')
ax.scatter(data[:,0], data[:,1], data[:,2],c=data[:,2], s=5)
ax.view_init(elev=10., azim=15)

ax = fig.add_subplot(2,3,4)
ax.scatter(data[:,0], data[:,1], c=data[:,2], s=10)
ax.set_ylabel('Y')
ax.set_xlabel('X')
ax = fig.add_subplot(2,3,5)
ax.scatter(data[:,0], data[:,2], c=data[:,2], s=10)
ax.set_xlabel('X')
ax.set_ylabel('Z')
ax = fig.add_subplot(2,3,6)
ax.scatter(data[:,1], data[:,2], c=data[:,2], s=10)
ax.set_xlabel('Y')
ax.set_ylabel('Z')

## Load the data

I have a recording with the headstage in 4 positions. Load them all and take the median position of the two lighthouses.

In [ ]:
coordinates = dict()
for name, folder in folders.items():
    fnames = os.listdir(data_root / folder)
    coordinates[name] = dict()
    for lindex in (1, 2):
        dat = [n for n in fnames if n.startswith('lighthouse_matrix%d'%lindex)]
        assert len(dat) == 1
        dat = dat[0]
        data = np.fromfile(data_root / folder / dat, dtype='double')
        data = data.reshape([-1, 3])
        # take the median
        data = np.nanmedian(data, axis=0)
        coordinates[name]['lighthouse%d'%lindex] = data
coordinates = pd.DataFrame(coordinates)

The two lighthouse signals (i.e. the two photodiode) should have constant distance, let's look at that

In [ ]:
delta = coordinates.loc['lighthouse1'] - coordinates.loc['lighthouse2']
av_delta = np.nanmedian([delta[d] for d in ('right', 'above', 'bottom')], axis=0)
std_delta = np.nanstd([delta[d] for d in ('right', 'above', 'bottom')], axis=0)
print(std_delta)

For the lighthouse2, the origin is nan. The data wasn't streamed at all. Replace that by what is expected from the delta measure

In [ ]:
coordinates.loc['lighthouse2', 'origin'] = coordinates.loc['lighthouse1', 'origin'] - av_delta
coordinates

## Subtract origin

First find the origin and subtract it.

In [ ]:
from copy import deepcopy

origin = coordinates.loc['lighthouse1', 'origin']
x1_pts = coordinates.loc['lighthouse1', 'right']
y1_pts = coordinates.loc['lighthouse1', 'bottom']
z1_pts = coordinates.loc['lighthouse1', 'above']
centered = deepcopy(coordinates.copy(deep=True))
for p in centered:
    centered.loc['lighthouse1', p] = centered.loc['lighthouse1', p]-origin
    centered.loc['lighthouse2', p] -= origin
centered

## Rotate

We have a new basis given by right, bottom, above. I want to change basis. 

For the vector right of origin for instance, we have
$\begin{bmatrix}
           x_{original} \\
           y_{original} \\
           z_{original}
  \end{bmatrix} =  \gamma_{right} \begin{bmatrix}
           1_{new} \\
           0_{new} \\
           0_{new}
  \end{bmatrix}
$, where $\gamma$ is the length of the new vector in real world unit. 

Which means that the new basis, $$\begin{bmatrix}
           1, 0, 0 \\
           0, 1, 0 \\
           0, 0, 1
  \end{bmatrix}_{new} = \begin{bmatrix}
           \frac{x_{right}}{\gamma_{right}}, \frac{x_{bottom}}{\gamma_{bottom}}, \frac{x_{above}}{\gamma_{above}} \\
           \frac{y_{right}}{\gamma_{right}}, \frac{y_{bottom}}{\gamma_{bottom}}, \frac{y_{above}}{\gamma_{above}} \\
           \frac{z_{right}}{\gamma_{right}}, \frac{z_{bottom}}{\gamma_{bottom}}, \frac{z_{above}}{\gamma_{above}},
  \end{bmatrix} = A$$

Then for any point, $\phi_{original} = A \phi_{new}$. This is trivially true for $\phi_{new} = (1, 0, 0)$, where  $\phi_{original}$ is the first column of $A$. Conversly, $\phi_{new} = A^{-1} \phi_{original}$

So I want to create a transformation matrix by concatenating the vector and dividing them by their length. Then invert it to get the transformation in the direction of interest.

In [ ]:
A_new2old = np.vstack(centered.loc['lighthouse1'])[1:,:].T
A_new2old[:,0] /= 10
A_new2old[:,1] /= 10
A_new2old[:,2] /= 8.14
print('New to old transformation matrix')
print(A_new2old)
A_old2new = np.matrix(np.linalg.inv(A_new2old))

print('\nOld to new transformation matrix')
print(A_old2new)


In [ ]:
# save the output:
np.savez(save_root / 'lighthouse_calibration.npz', origin=origin, A_light2world=A_old2new, A_world2light=A_new2old)

In [ ]:
centered.loc['lighthouse2', p]

In [ ]:
# Trivial check:
for p in centered:
    print('Original vector for %s lighthouse1: %s' % (p, centered.loc['lighthouse1', p]))
    print('    In new coordinates: %s' % (np.round(np.dot(A_old2new, centered.loc['lighthouse1', p]), 3)))
print('\n')
    
for p in centered:
    print('Original vector for %s lighthouse2: %s' % (p, centered.loc['lighthouse2', p]))
    print('    In new coordinates: %s' % (np.round(np.dot(A_old2new, centered.loc['lighthouse2', p]), 3)))

# Use it

Load an example file and apply the transform

In [ ]:
# load the data
ex_dir = data_root / '..' / 'example_2boxes'
data = np.fromfile(ex_dir / 'lighthouse_matrix2_2021-09-27T12_08_25.dat', dtype='double')
data = data.reshape([-1, 3])

# center
cdata = data - origin

# rotate
rdata = np.array(np.matmul(A_old2new, cdata.T)).T

In [ ]:
fig = plt.figure(figsize=(10,6))

ax = fig.add_subplot(2,3,1, aspect='equal')
ax.set_title('Original x/y')
ax.scatter(data[:,0], data[:,1], c=data[:,2], s=1)
ax = fig.add_subplot(2,3,2)
ax.set_title('Original x/z')
ax.scatter(data[:,0], data[:,2], c=data[:,2], s=1)
ax = fig.add_subplot(2,3,3)
ax.set_title('Original y/z')
ax.scatter(data[:,1], data[:,2], c=data[:,2], s=1)


ax = fig.add_subplot(2,3,4, aspect='equal')
ax.set_title('New x/y')
ax.scatter(rdata[:,0], rdata[:,1], c=rdata[:,2], s=1)
ax.set_xlim(rdata[:,0].min(), rdata[:,0].max())
ax = fig.add_subplot(2,3,5)
ax.set_title('New x/z')
ax.scatter(rdata[:,0], rdata[:,2], c=rdata[:,2], s=1)
ax = fig.add_subplot(2,3,6)
ax.set_title('New y/z')
ax.scatter(rdata[:,1], rdata[:,2], c=rdata[:,2], s=1)

# Transform real recording

In [ ]:
# same with a longer recording with the mouse
example_dir = data_root / '..' / 'tracking' / 'mouse_in_arena_210928'

data_files = ('lighthouse_matrix1_2021-09-28T14_17_34.dat',
              'lighthouse_matrix2_2021-09-28T14_17_34.dat')

matrix = []
for fname in data_files:
    data = np.fromfile(example_dir/fname, dtype='double')
    data = data.reshape([-1,3])
    print(data.shape)
    matrix.append(data)
data = np.vstack(matrix)
data.shape

# center
cdata = data - origin

# rotate
rdata = np.array(np.matmul(A_old2new, cdata.T)).T
rdata2 = np.matmul(data, A_old2new)

In [ ]:
fig = plt.figure(figsize=(10,5))

label = ['x','y', 'z']
for iax, (x, y) in enumerate([(0,1), (0,2), (1,2)]):
    ax = fig.add_subplot(2,3,1+iax)
    ax.scatter(data[:,x], data[:,y], c=data[:,2], s=10)
    for pts, color in zip([x1_pts, y1_pts, z1_pts], ['b','k', 'r']):
        ax.plot([origin[x], pts[x]], [origin[y], pts[y]], color=color)
    ax.set_ylabel(label[y])
    ax.set_xlabel(label[x])
  
for iax, (x, y) in enumerate([(0,1), (0,2), (1,2)]):
    ax = fig.add_subplot(2,3,4+iax)
    ax.scatter(rdata[:,x], rdata[:,y], c=rdata[:,2], s=10)
    for pts_o, color in zip([x1_pts, y1_pts, z1_pts], ['b','k', 'r']):
        pts = A_old2new * (pts_o - origin)[:, np.newaxis]
        ax.plot([0, pts[x]], [0, pts[y]], color=color)
    ax.set_ylabel(label[y])
    ax.set_xlabel(label[x])
    